# Vapor Liquid Liquid Equilibrium (VLLE)

This notebook exemplifies the three phase equilibria calculation. The VLLE calculation is separated into two cases:

- Binary mixtures: due to degrees of freedom restrictions the VLLE is computed at given temperature or pressure. This is solved using the  ``vlleb`` function.

- Mixtures with three or more components: the VLLE is computed at given global composition, temperature and pressure. This is solved using the ``vlle`` function.


To start, the required functions are imported.

In [1]:
import numpy as np
from sgtpy import component, mixture, saftvrmie
from sgtpy.equilibrium import vlle, vlleb

---
## Binary (two-component) mixture VLLE

The VLLE computation for binary mixtures is solution is based on the following objective function:


$$ K_{ik} x_{ir} - x_{ik} = 0  \qquad i = 1,...,c \quad k = 2,3 $$
$$ \sum_{i=1}^c x_{ik} = 1 \qquad k = 1, 2, 3$$

Where, $x_{ik}$ is the molar fraction of the component $i$ on the phase $k$ and $ K_{ik} = x_{ik}/x_{ir} = \hat{\phi}_{ir}/\hat{\phi}_{ik} $ is the constant equilibrium respect to a reference phase $r$. 


**note:** this calculation does not check for the stability of the phases.


In the following code block, the VLLE calculation for the binary mixture of water and butanol is exemplified.   For binary mixtures, the VLLE is computed at either given pressure (``P``) or temperature (``T``). The function ``vlleb`` requires either of those and initial guesses unknown variables (phase compositions and unknown specification).

First, the mixture and its interaction parameters are set up.

In [2]:
# creating pure components
water = component('water', ms = 1.7311, sigma = 2.4539 , eps = 110.85,
                    lambda_r = 8.308, lambda_a = 6., eAB = 1991.07, rcAB = 0.5624,
                    rdAB = 0.4, sites = [0,2,2], cii = 1.5371939421515458e-20)

butanol = component('butanol2C', ms = 1.9651, sigma = 4.1077 , eps = 277.892,
                    lambda_r = 10.6689, lambda_a = 6., eAB = 3300.0, rcAB = 0.2615,
                    rdAB = 0.4, sites = [1,0,1], npol = 1.45, mupol = 1.6609,
                    cii  = 1.5018715324070352e-19)
mix = mixture(water, butanol)
# or
mix = water + butanol

# optimized from experimental LLE
kij, lij = np.array([-0.00736075, -0.00737153])
Kij = np.array([[0, kij], [kij, 0]])
Lij = np.array([[0., lij], [lij, 0]])

# setting interactions corrections
mix.kij_saft(Kij)
mix.lij_saft(Lij)
# creating eos model
eosb = saftvrmie(mix)

The binary VLLE at constant pressure is computed as follows:

In [3]:
#computed 
P = 1.01325e5 # Pa
# initial guesses
x0 = np.array([0.96, 0.06])
w0 = np.array([0.53, 0.47])
y0 = np.array([0.8, 0.2])
T0 = 350. # K
vlleb(x0, w0, y0, T0, P, 'P', eosb, full_output=True)

      T: 367.204561868693
      P: 101325.0
  error: 7.323596930421064e-10
   nfev: 17
      X: array([0.94267072, 0.05732928])
     vx: 2.2876713199773015e-05
  Xassx: array([0.1189557 , 0.14314252, 0.05083917, 0.10609223])
 statex: 'Liquid'
      W: array([0.61296236, 0.38703764])
     vw: 4.8943569001653434e-05
  Xassw: array([0.14552605, 0.27708904, 0.08764041, 0.22462698])
 statew: 'Liquid'
      Y: array([0.77529986, 0.22470014])
     vy: 0.029275241724589694
  Xassy: array([0.98580533, 0.98766678, 0.97137887, 0.98550529])
 statey: 'Vapor'

Similarly, the binary VLLE at constant temperature is computed below:

In [4]:
T = 350. # K
# initial guesses
x0 = np.array([0.96, 0.06])
w0 = np.array([0.53, 0.47])
y0 = np.array([0.8, 0.2])
P0 = 4e4 # Pa
sol_vlleb = vlleb(x0, w0, y0, P0, T, 'T', eosb, full_output=True)
sol_vlleb

      T: 350.0
      P: 50892.230927675
  error: 5.990856155728776e-12
   nfev: 17
      X: array([0.94825583, 0.05174417])
     vx: 2.2088195154118556e-05
  Xassx: array([0.10141378, 0.12353259, 0.03996736, 0.08493912])
 statex: 'Liquid'
      W: array([0.58993921, 0.41006079])
     vw: 4.976651311789804e-05
  Xassw: array([0.12269929, 0.25903408, 0.07057308, 0.19750628])
 statew: 'Liquid'
      Y: array([0.79056736, 0.20943264])
     vy: 0.05606543054348498
  Xassy: array([0.99018342, 0.99145345, 0.97859919, 0.98919639])
 statey: 'Vapor'

You can also supply initial guesses for the phase volumes (``v0``) or non-bonded association site fractions (``Xass0``), which can come from a previous calculation using the ``full_output=True`` option.

In [5]:
T = 350. # K
# initial guesses
x0 = np.array([0.96, 0.06])
w0 = np.array([0.53, 0.47])
y0 = np.array([0.8, 0.2])
P0 = 4e4 # Pa
v0 = [sol_vlleb.vx, sol_vlleb.vw, sol_vlleb.vy]
Xass0 = [sol_vlleb.Xassx, sol_vlleb.Xassx, sol_vlleb.Xassx]
# VLLE supplying initial guess for volumes and non-bonded association sites fractions
vlleb(x0, w0, y0, P0, T, 'T', eosb, v0=v0, Xass0=Xass0, full_output=True)

      T: 350.0
      P: 50892.230927673845
  error: 5.978882719897937e-12
   nfev: 17
      X: array([0.94825583, 0.05174417])
     vx: 2.208819515411813e-05
  Xassx: array([0.10141378, 0.12353259, 0.03996736, 0.08493912])
 statex: 'Liquid'
      W: array([0.58993921, 0.41006079])
     vw: 4.976651311789815e-05
  Xassw: array([0.12269929, 0.25903408, 0.07057308, 0.19750628])
 statew: 'Liquid'
      Y: array([0.79056736, 0.20943264])
     vy: 0.05606543054348634
  Xassy: array([0.99018342, 0.99145345, 0.97859919, 0.98919639])
 statey: 'Vapor'

---
## Multicomponent mixture VLLE

Phase stability plays a key role during equilibrium computation when dealing with more than two liquid phases. For this purpose the following modified multiphase Rachford-Rice mass balance has been proposed [Gupta et al.](https://www.sciencedirect.com/science/article/pii/037838129180021M):


$$ \sum_{i=1}^c \frac{z_i (K_{ik} \exp{\theta_k}-1)}{1+ \sum\limits^{\pi}_{\substack{j=1 \\ j \neq r}}{\psi_j (K_{ij}} \exp{\theta_j} -1)} = 0 \qquad k = 1,..., \pi,  k \neq r $$

Subject to:

$$ \psi_k \theta_k = 0 $$

In this system of equations, $z_i$ represents the global composition of the component $i$,  $ K_{ij} = x_{ij}/x_{ir} = \hat{\phi}_{ir}/\hat{\phi}_{ij} $ is the constant equilibrium of component $i$ in phase $j$ respect to the reference phase $r$, and $\psi_j$ and $\theta_j$ are the phase fraction and stability variable of the phase $j$.  

The solution strategy is similar to the classic isothermal isobaric two-phase flash. First, a reference phase must be selected, this phase is considered stable during the procedure. In an inner loop, the system of equations is solved using multidimensional Newton's method for phase fractions and stability variables and then compositions are updated in an outer loop using accelerated successive substitution (ASS).  Once the algorithm has converged, the stability variable gives information about the phase. If it takes a value of zero the phase is stable and if it is positive the phase is not.  The proposed successive substitution method can be slow, if that is the case the algorithm attempts to minimize Gibbs Free energy of the system. This procedure also ensures stable solutions and is solved using SciPy's functions.

$$ min \, {G} = \sum_{k=1}^\pi \sum_{i=1}^c F_{ik} \ln \hat{f}_{ik}  $$


The next code block exemplifies the VLLE calculation for the mixture of water, ethanol and MTBE. This is done with the ``vlle`` function, which incorporates the algorithm described above. This functions requires the global composition (``z``), temperature (``T``) and pressure (``P``). Additionally, the ``vlle`` function requires initial guesses for the composition of the phases.

In [6]:
water = component('water', ms = 1.7311, sigma = 2.4539 , eps = 110.85,
                    lambda_r = 8.308, lambda_a = 6.,  eAB = 1991.07, rcAB = 0.5624,
                    rdAB = 0.4, sites = [0,2,2], cii = 1.5371939421515455e-20)

butanol = component('butanol2C', ms = 1.9651, sigma = 4.1077 , eps = 277.892,
                    lambda_r = 10.6689, lambda_a = 6., eAB = 3300.0, rcAB = 0.2615,
                    rdAB = 0.4, sites = [1,0,1], npol = 1.45, mupol = 1.6609,
                    cii  = 1.5018715324070352e-19)

mtbe = component('mtbe', ms =2.17847383,  sigma=  4.19140014, eps =  306.52083841,
                 lambda_r = 14.74135198, lambda_a = 6.0, npol = 2.95094686,  
                 mupol = 1.3611, sites = [0,0,1], cii =3.5779968517655445e-19 )

mix = mixture(water, butanol)
mix.add_component(mtbe)
# or

mix = water + butanol + mtbe


#butanol water
k12, l12 = np.array([-0.00736075, -0.00737153])

#mtbe butanol
k23 = -0.0029995
l23 = 0.
rc23 =  1.90982649

#mtbe water
k13 = -0.07331438
l13 = 0.
rc13 = 2.84367922

# setting up interaction corrections
Kij = np.array([[0., k12, k13], [k12, 0., k23], [k13, k23, 0.]])
Lij = np.array([[0., l12, l13], [l12, 0., l23], [l13, l23, 0.]])
mix.kij_saft(Kij)
mix.lij_saft(Lij)
eos = saftvrmie(mix)

# setting up induced association
#mtbe water
eos.eABij[0,2] = water.eAB / 2
eos.eABij[2,0] = water.eAB / 2
eos.rcij[0,2] = rc13 * 1e-10
eos.rcij[2,0] = rc13 * 1e-10
#mtbe butanol
eos.eABij[2,1] = butanol.eAB / 2
eos.eABij[1,2] = butanol.eAB / 2
eos.rcij[2,1] = rc23 * 1e-10
eos.rcij[1,2] = rc23 * 1e-10

Once the ternary mixture has been set up, the VLLE computation is performed as follows:

In [7]:
T = 345. #K
P = 1.01325e5 # Pa
# global composition
z = np.array([0.5, 0.3, 0.2])
# initial guesses
x0 = np.array([0.9, 0.05, 0.05])
w0 = np.array([0.45, 0.45, 0.1])
y0 = np.array([0.3, 0.1, 0.6])
sol_vlle = vlle(x0, w0, y0, z, T, P, eos, full_output = True)
sol_vlle

           T: 345.0
           P: 101325.0
 error_outer: 2.3398050816014838e-11
 error_inner: 2.0735459318708586e-10
        iter: 21
        beta: array([0.14191476, 0.70782655, 0.15025869])
       tetha: array([0., 0., 0.])
           X: array([[0.96430193, 0.0305612 , 0.00513686],
       [0.44366261, 0.40404876, 0.15228863],
       [0.32687062, 0.06433221, 0.60879717]])
           v: [2.0897688177998606e-05, 6.500650541939807e-05, 0.027607765914231478]
        Xass: [array([0.09661167, 0.11235917, 0.0360625 , 0.07512566, 0.11881848]), array([0.12710994, 0.33397064, 0.08224464, 0.26132655, 0.39334806]), array([0.98662764, 0.99250597, 0.97757231, 0.99022329, 0.9943016 ])]
      states: ['L', 'L', 'V']

As for the other phase equilibria functions included, you can supply initial guesses for the phase volumes (``v0``) or non-bonded association site fractions (``Xass0``), which can come from a previous calculation using the ``full_output=True`` option.

In [8]:
T = 345. #K
P = 1.01325e5 # Pa
# global composition
z = np.array([0.5, 0.3, 0.2])
# initial guesses
x0 = np.array([0.9, 0.05, 0.05])
w0 = np.array([0.45, 0.45, 0.1])
y0 = np.array([0.3, 0.1, 0.6])
v0 = sol_vlle.v
Xass0 = sol_vlle.Xass
# VLLE supplying initial guess for volumes and non-bonded association sites fractions
vlle(x0, w0, y0, z, T, P, eos, v0=v0, Xass0=Xass0, full_output = True)

           T: 345.0
           P: 101325.0
 error_outer: 2.3398055623895206e-11
 error_inner: 2.0735462775123235e-10
        iter: 21
        beta: array([0.14191476, 0.70782655, 0.15025869])
       tetha: array([0., 0., 0.])
           X: array([[0.96430193, 0.0305612 , 0.00513686],
       [0.44366261, 0.40404876, 0.15228863],
       [0.32687062, 0.06433221, 0.60879717]])
           v: [2.0897688177999365e-05, 6.500650541936502e-05, 0.027607765914231426]
        Xass: [array([0.09661167, 0.11235917, 0.0360625 , 0.07512566, 0.11881848]), array([0.12710994, 0.33397064, 0.08224464, 0.26132655, 0.39334806]), array([0.98662764, 0.99250597, 0.97757231, 0.99022329, 0.9943016 ])]
      states: ['L', 'L', 'V']

---
For further information about these functions check out the documentation running: ``vlleb?`` or ``vlle?``